In [1]:
import lightgbm as lgb
import numpy as np
from os.path import join
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna.integration.lightgbm as optuna_lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
import math
from operator import itemgetter
import itertools
from sklearn.model_selection import TimeSeriesSplit


# https://gist.github.com/vbkaisetsu/d7c08e9c5aabe13686dd554ddfadf076
def suffix_array(arr):
    arr_size = len(arr)
    arr_int = {v: k for k, v in enumerate(sorted(set(arr)))}
    arr = [arr_int[x] for x in arr]
    arr.append(-1)
    suf = [[i, arr[i], arr[i + 1]] for i in range(arr_size)]
    suf.sort(key=itemgetter(1, 2))
    idx = [0] * arr_size
    k = 2
    while k < arr_size:
        r = 0
        prev_r = suf[0][1]
        for i in range(arr_size):
            if suf[i][1] != prev_r or suf[i - 1][2] != suf[i][2]:
                r += 1
            prev_r = suf[i][1]
            suf[i][1] = r
            idx[suf[i][0]] = i
        for i in range(arr_size):
            next_idx = suf[i][0] + k
            suf[i][2] = suf[idx[next_idx]][1] if next_idx < arr_size else -1
        suf.sort(key=itemgetter(1, 2))
        k <<= 1
    return [x[0] for x in suf]


# https://gist.github.com/vbkaisetsu/d7c08e9c5aabe13686dd554ddfadf076
def bwt_encode(data):
    data_ref = suffix_array(data)
    bwt_ref = (x - 1 for x in data_ref)
    return ''.join([data[x] for x in bwt_ref])


def balanced_bwt_rl_entropy(text):
    rl_len = len([ch for ch, _ in itertools.groupby(bwt_encode(text))])
    return math.exp(len(text) / rl_len)


data_dir = '../data'
train_data_path = join(data_dir, 'train.parquet')


def loadData():
    df = pd.read_parquet(train_data_path)
    df.drop(columns=[
        'id', 'authorName', 'authorPhoto', 'authorChannelId', 'timestampUsec',
        'originChannelId', 'originVideoId', 'membership', 'purchase',
        'rawMessage', 'message'
    ],
            inplace=True)
    return df



In [2]:

data = loadData()

grp = data.groupby('spam')
grp = grp.apply(lambda x: x.sample(grp.size().min()))
data = pd.DataFrame(grp).reset_index(drop=True)
print(len(data))

x = data.drop(columns='spam')
y = data['spam']
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
# print(x_train.head())
# print(y_train.head())
# validation_data = lgb.Dataset(x_test, label=y_test)


2814


In [3]:

train_data = lgb.Dataset(x, label=y)
tscv = TimeSeriesSplit(n_split=5)
param = {
    'device_type': 'gpu',
    'objective': 'binary',
    'boosting': 'gbdt',
    # 'metric': 'kullback_leibler',
    # 'metric': 'auc',
    # 'is_unbalance': True,
}

bst = optuna_lgb.cv(param, train_data, folds=tscv)
bst.save_model('model.txt')


TypeError: __init__() got an unexpected keyword argument 'n_split'

In [4]:

train_data = lgb.Dataset(x, label=y)
tscv = TimeSeriesSplit()
param = {
    'device_type': 'gpu',
    'objective': 'binary',
    'boosting': 'gbdt',
    # 'metric': 'kullback_leibler',
    # 'metric': 'auc',
    # 'is_unbalance': True,
}

bst = optuna_lgb.cv(param, train_data, folds=tscv)
bst.save_model('model.txt')


hat meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped tra

AttributeError: 'dict' object has no attribute 'save_model'

In [5]:

train_data = lgb.Dataset(x, label=y)
tscv = TimeSeriesSplit()
param = {
    'device_type': 'gpu',
    'objective': 'binary',
    'boosting': 'gbdt',
    # 'metric': 'kullback_leibler',
    # 'metric': 'auc',
    # 'is_unbalance': True,
}

bst = optuna_lgb.cv(param, train_data)
bst.save_model('model.txt')


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

In [6]:

train_data = lgb.Dataset(x, label=y)
tscv = TimeSeriesSplit()
param = {
    'device_type': 'gpu',
    'objective': 'binary',
    'boosting': 'gbdt',
    # 'metric': 'kullback_leibler',
    # 'metric': 'auc',
    # 'is_unbalance': True,
}

bst = optuna_lgb.cv(param, train_data)
bst.save_model('model.txt')


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

In [7]:

train_data = lgb.Dataset(x, label=y)
tscv = TimeSeriesSplit()
param = {
    'device_type': 'gpu',
    'objective': 'binary',
    'boosting': 'gbdt',
    # 'metric': 'kullback_leibler',
    # 'metric': 'auc',
    # 'is_unbalance': True,
}
bst = lgb.cv(param, train_data)
bst.save_model('model.txt')


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

In [8]:

train_data = lgb.Dataset(x, label=y)
tscv = TimeSeriesSplit()
param = {
    'device_type': 'gpu',
    'application': 'binary',
    'boosting': 'gbdt',
    # 'metric': 'kullback_leibler',
    # 'metric': 'auc',
    # 'is_unbalance': True,
}
bst = lgb.cv(param, train_data)
bst.save_model('model.txt')


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

In [9]:

train_data = lgb.Dataset(x, label=y)
tscv = TimeSeriesSplit()
param = {
    'device_type': 'gpu',
    'application': 'binary',
    'objective': 'binary',
    'boosting': 'gbdt',
    # 'metric': 'kullback_leibler',
    # 'metric': 'auc',
    # 'is_unbalance': True,
}
bst = lgb.cv(param, train_data)
bst.save_model('model.txt')


[LightGBM] [Warning] objective is set=binary, application=binary will be ignored. Current value: objective=binary


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

In [10]:

train_data = lgb.Dataset(x, label=y)
tscv = TimeSeriesSplit()
param = {
    'device_type': 'gpu',
    'application': 'binary',
    'objective': 'binary',
    'boosting': 'gbdt',
    # 'metric': 'kullback_leibler',
    # 'metric': 'auc',
    # 'is_unbalance': True,
}
bst = lgb.cv(param, train_data, stratified=True)
bst.save_model('model.txt')


[LightGBM] [Warning] objective is set=binary, application=binary will be ignored. Current value: objective=binary


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

In [11]:

train_data = lgb.Dataset(x, label=y)
tscv = TimeSeriesSplit()
param = {
    'device_type': 'gpu',
    'application': 'binary',
    'objective': 'binary',
    'boosting': 'gbdt',
    # 'metric': 'kullback_leibler',
    # 'metric': 'auc',
    # 'is_unbalance': True,
}
bst = lgb.cv(param, train_data, stratified=False)
bst.save_model('model.txt')


[LightGBM] [Warning] objective is set=binary, application=binary will be ignored. Current value: objective=binary
[LightGBM] [Warning] objective is set=binary, application=binary will be ignored. Current value: objective=binary
[LightGBM] [Warning] objective is set=binary, application=binary will be ignored. Current value: objective=binary
[LightGBM] [Info] Number of positive: 1502, number of negative: 746
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 196270
[LightGBM] [Info] Number of data points in the train set: 2248, number of used features: 773
[LightGBM] [Info] Using GPU Device: GeForce GTX TITAN X, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 772 dense feature groups (1.66 MB) transferred to GPU in 0.011914 secs. 1 sparse feature groups
[LightGBM] [Warning] objective is set=binary, application=binary

AttributeError: 'dict' object has no attribute 'save_model'

In [12]:
bst

{'binary_logloss-mean': [0.5419050144040132,
  0.4678303486639841,
  0.40766557424529504,
  0.35759710686370194,
  0.3152334107746819,
  0.2789936203600265,
  0.2477239326900103,
  0.2205311922207521,
  0.19676724985788857,
  0.1758975159446327,
  0.15748384461646853,
  0.1412029826173551,
  0.12676307945210202,
  0.11392718371784691,
  0.10250583265339212,
  0.09230556087638166,
  0.08320573867722367,
  0.07505542444115616,
  0.06775406354010602,
  0.061212839259827456,
  0.05533697611487011,
  0.050067197135826746,
  0.04533716807942591,
  0.04107794435593198,
  0.037247179943606115,
  0.033800760702800345,
  0.03069872728548404,
  0.027898752773622103,
  0.025382539241594164,
  0.023101971703033033,
  0.021059066436211613,
  0.019211651025470698,
  0.017542998626524472,
  0.016041413229348447,
  0.014698244633537225,
  0.013477037176613384,
  0.012385149007592824,
  0.011392986757127655,
  0.010504648645798021,
  0.00969426656877272,
  0.0089548163937035,
  0.008293090334995219,
  0

In [13]:
bst.keys()

dict_keys(['binary_logloss-mean', 'binary_logloss-stdv'])

In [14]:

train_data = lgb.Dataset(x, label=y)
tscv = TimeSeriesSplit()
param = {
    'device_type': 'gpu',
    'objective': 'binary',
    'boosting': 'gbdt',
    # 'metric': 'kullback_leibler',
    # 'metric': 'auc',
    # 'is_unbalance': True,
}
bst = lgb.cv(param, train_data, stratified=False, return_cvbooster=True)
bst.save_model('model.txt')


[LightGBM] [Info] Number of positive: 1502, number of negative: 746
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 196270
[LightGBM] [Info] Number of data points in the train set: 2248, number of used features: 773
[LightGBM] [Info] Using GPU Device: GeForce GTX TITAN X, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 772 dense feature groups (1.66 MB) transferred to GPU in 0.008896 secs. 1 sparse feature groups
[LightGBM] [Info] Number of positive: 1506, number of negative: 742
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 196270
[LightGBM] [Info] Number of data points in the train set: 2248, number of used features: 773
[LightGBM] [Info] Using GPU Device: GeForce GTX TITAN X, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU 

AttributeError: 'dict' object has no attribute 'save_model'

In [15]:
bst.keys()

dict_keys(['binary_logloss-mean', 'binary_logloss-stdv', 'cvbooster'])

In [16]:
bst.cvbooster

AttributeError: 'dict' object has no attribute 'cvbooster'

In [17]:
bst['cvbooster']

In [18]:
bst['cvbooster'].save_model

<function lightgbm.engine.CVBooster.__getattr__.<locals>.handler_function(*args, **kwargs)>

In [19]:

train_data = lgb.Dataset(x, label=y)
tscv = TimeSeriesSplit()
param = {
    'device_type': 'gpu',
    'objective': 'binary',
    'boosting': 'gbdt',
    # 'metric': 'kullback_leibler',
    # 'metric': 'auc',
    # 'is_unbalance': True,
}
bst = lgb.cv(param, train_data, stratified=False,
             return_cvbooster=True)['cvbooster']
bst.save_model('model.txt')


[LightGBM] [Info] Number of positive: 1502, number of negative: 746
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 196270
[LightGBM] [Info] Number of data points in the train set: 2248, number of used features: 773
[LightGBM] [Info] Using GPU Device: GeForce GTX TITAN X, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 772 dense feature groups (1.66 MB) transferred to GPU in 0.007596 secs. 1 sparse feature groups
[LightGBM] [Info] Number of positive: 1506, number of negative: 742
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 196270
[LightGBM] [Info] Number of data points in the train set: 2248, number of used features: 773
[LightGBM] [Info] Using GPU Device: GeForce GTX TITAN X, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU 

In [20]:

train_data = lgb.Dataset(x, label=y)

param = {
    'device_type': 'gpu',
    'objective': 'binary',
    'boosting': 'gbdt',
    "verbosity": -1,
    # 'metric': 'kullback_leibler',
    'metric': 'binary_logloss',
    # 'is_unbalance': True,
}
bst = optuna_lgb.cv(param,
                    train_data,
                    stratified=False,
                    verbose_eval=100,
                    early_stopping_rounds=100,
                    folds=KFold(n_splits=3))
bst.save_model('model.txt')


NameError: name 'KFold' is not defined

In [21]:
import lightgbm as lgb
import numpy as np
from os.path import join
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna.integration.lightgbm as optuna_lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
import math
from operator import itemgetter
import itertools
from sklearn.model_selection import KFold


# https://gist.github.com/vbkaisetsu/d7c08e9c5aabe13686dd554ddfadf076
def suffix_array(arr):
    arr_size = len(arr)
    arr_int = {v: k for k, v in enumerate(sorted(set(arr)))}
    arr = [arr_int[x] for x in arr]
    arr.append(-1)
    suf = [[i, arr[i], arr[i + 1]] for i in range(arr_size)]
    suf.sort(key=itemgetter(1, 2))
    idx = [0] * arr_size
    k = 2
    while k < arr_size:
        r = 0
        prev_r = suf[0][1]
        for i in range(arr_size):
            if suf[i][1] != prev_r or suf[i - 1][2] != suf[i][2]:
                r += 1
            prev_r = suf[i][1]
            suf[i][1] = r
            idx[suf[i][0]] = i
        for i in range(arr_size):
            next_idx = suf[i][0] + k
            suf[i][2] = suf[idx[next_idx]][1] if next_idx < arr_size else -1
        suf.sort(key=itemgetter(1, 2))
        k <<= 1
    return [x[0] for x in suf]


# https://gist.github.com/vbkaisetsu/d7c08e9c5aabe13686dd554ddfadf076
def bwt_encode(data):
    data_ref = suffix_array(data)
    bwt_ref = (x - 1 for x in data_ref)
    return ''.join([data[x] for x in bwt_ref])


def balanced_bwt_rl_entropy(text):
    rl_len = len([ch for ch, _ in itertools.groupby(bwt_encode(text))])
    return math.exp(len(text) / rl_len)


data_dir = '../data'
train_data_path = join(data_dir, 'train.parquet')


def loadData():
    df = pd.read_parquet(train_data_path)
    df.drop(columns=[
        'id', 'authorName', 'authorPhoto', 'authorChannelId', 'timestampUsec',
        'originChannelId', 'originVideoId', 'membership', 'purchase',
        'rawMessage', 'message'
    ],
            inplace=True)
    return df



In [22]:

train_data = lgb.Dataset(x, label=y)

param = {
    'device_type': 'gpu',
    'objective': 'binary',
    'boosting': 'gbdt',
    "verbosity": -1,
    # 'metric': 'kullback_leibler',
    'metric': 'binary_logloss',
    # 'is_unbalance': True,
}
bst = optuna_lgb.cv(param,
                    train_data,
                    stratified=False,
                    verbose_eval=100,
                    early_stopping_rounds=100,
                    folds=KFold(n_splits=3))
bst.save_model('model.txt')


[100]	cv_agg's binary_logloss: 15.0778 + 14.437


AttributeError: 'dict' object has no attribute 'save_model'

In [23]:

train_data = lgb.Dataset(x, label=y)

param = {
    'device_type': 'gpu',
    'objective': 'binary',
    'boosting': 'gbdt',
    "verbosity": -1,
    # 'metric': 'kullback_leibler',
    'metric': 'binary_logloss',
    # 'is_unbalance': True,
}
tuner = optuna_lgb.LightGBMTunerCV(param,
                                   train_data,
                                   stratified=False,
                                   verbose_eval=100,
                                   early_stopping_rounds=100,
                                   folds=KFold(n_splits=3))
tuner.run()
bst = tuner.get_best_booster()
bst.save_model('model.txt')


]	cv_agg's binary_logloss: 15.0778 + 14.437
[100]	cv_agg's binary_logloss: 15.0778 + 14.437
min_data_in_leaf, val_score: 11.978138: 100%|##########| 5/5 [00:19<00:00,  3.91s/it][100]	cv_agg's binary_logloss: 15.0778 + 14.437



ValueError: LightGBMTunerCV requires `return_cvbooster=True` for method `get_best_booster()`.

In [24]:

train_data = lgb.Dataset(x, label=y)

param = {
    'device_type': 'gpu',
    'objective': 'binary',
    'boosting': 'gbdt',
    "verbosity": -1,
    # 'metric': 'kullback_leibler',
    'metric': 'binary_logloss',
    # 'is_unbalance': True,
}
tuner = optuna_lgb.LightGBMTunerCV(param,
                                   train_data,
                                   stratified=False,
                                   verbose_eval=100,
                                   early_stopping_rounds=100,
                                   return_cvbooster=True,
                                   folds=KFold(n_splits=3, shuffle=True))
tuner.run()
bst = tuner.get_best_booster()
bst.save_model('model.txt')


agg's binary_logloss: 0.00472714 + 0.00471799
regularization_factors, val_score: 0.003061:  35%|###5      | 7/20 [00:12<00:21,  1.69s/it][100]	cv_agg's binary_logloss: 0.00420607 + 0.00294259
[200]	cv_agg's binary_logloss: 0.00359118 + 0.00367785
[300]	cv_agg's binary_logloss: 0.00354504 + 0.00403509
regularization_factors, val_score: 0.003061:  40%|####      | 8/20 [00:17<00:33,  2.79s/it][I 2021-01-22 16:27:41,600] Trial 47 finished with value: 0.003515071360347013 and parameters: {'lambda_l1': 1.1813844325780188e-06, 'lambda_l2': 2.1915359970266306}. Best is trial 41 with value: 0.0030707240654798493.
[400]	cv_agg's binary_logloss: 0.00355061 + 0.00427969
regularization_factors, val_score: 0.003061:  70%|#######   | 14/20 [00:27<00:10,  1.70s/it][100]	cv_agg's binary_logloss: 0.00912207 + 0.00221378
[200]	cv_agg's binary_logloss: 0.00884114 + 0.00226544
[300]	cv_agg's binary_logloss: 0.00878322 + 0.00227731
[400]	cv_agg's binary_logloss: 0.00872338 + 0.00227159
[500]	cv_agg's binary

In [25]:
msg = [
    'HEEERRREESSS SUISEI~~~', 'wwwww', 'なにそれ', 'YABE', 'やばい', '草', 'lol why',
    'クソクソクソ', '死ね死ね死ね死ね死ね死ね死ね死ね死ね死ね死ね死ね',
    'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', 'きもきもきも',
    'パピオン加藤最強!!パピオン加藤最強!!!パピオン加藤最強!!パピオン加藤最強!!!'
]
bwt = np.nan
test_data = [[
    0.0, 0.0, 0.0, 0.0, 0.0, 6,
    len(x),
    len(set(x)),
    balanced_bwt_rl_entropy([x])
] for x in msg]
print(test_data)
test_df = pd.DataFrame(test_data,
                       columns=[
                           'isVerified', 'isOwner', 'isModerator', 'isMember',
                           'isSuperchat', 'authorLength', 'messageLength',
                           'messageUniqueness', 'bwtrl'
                       ])
embeds = model.encode(msg)
emb_columns = ['emb_' + str(i) for i in range(embeds.shape[1])]
test_df[emb_columns] = embeds
test_df['isMember'] = test_df['isMember']
test_df['isSuperchat'] = test_df['isSuperchat']
sam = data[data['spam'] == 1.0].sample(10)
val_pred = bst.predict(test_df)
print(val_pred)
print((val_pred > 0.65).astype(int))

[[0.0, 0.0, 0.0, 0.0, 0.0, 6, 22, 8, 1.0799589994281866], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 5, 1, 2.718281828459045], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 4, 4, 1.2840254166877414], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 4, 4, 1.2840254166877414], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 3, 3, 1.3956124250860895], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 1, 1, 2.718281828459045], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 7, 6, 1.1535649948951077], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 6, 2, 1.1813604128656459], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 24, 2, 1.0425469051899914], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 59, 1, 2.718281828459045], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 6, 2, 1.1813604128656459], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 42, 9, 1.0281671774377144]]


NameError: name 'model' is not defined

In [26]:

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')


In [27]:
msg = [
    'HEEERRREESSS SUISEI~~~', 'wwwww', 'なにそれ', 'YABE', 'やばい', '草', 'lol why',
    'クソクソクソ', '死ね死ね死ね死ね死ね死ね死ね死ね死ね死ね死ね死ね',
    'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', 'きもきもきも',
    'パピオン加藤最強!!パピオン加藤最強!!!パピオン加藤最強!!パピオン加藤最強!!!'
]
bwt = np.nan
test_data = [[
    0.0, 0.0, 0.0, 0.0, 0.0, 6,
    len(x),
    len(set(x)),
    balanced_bwt_rl_entropy([x])
] for x in msg]
print(test_data)
test_df = pd.DataFrame(test_data,
                       columns=[
                           'isVerified', 'isOwner', 'isModerator', 'isMember',
                           'isSuperchat', 'authorLength', 'messageLength',
                           'messageUniqueness', 'bwtrl'
                       ])
embeds = model.encode(msg)
emb_columns = ['emb_' + str(i) for i in range(embeds.shape[1])]
test_df[emb_columns] = embeds
test_df['isMember'] = test_df['isMember']
test_df['isSuperchat'] = test_df['isSuperchat']
sam = data[data['spam'] == 1.0].sample(10)
val_pred = bst.predict(test_df)
print(val_pred)
print((val_pred > 0.65).astype(int))

[[0.0, 0.0, 0.0, 0.0, 0.0, 6, 22, 8, 1.0799589994281866], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 5, 1, 2.718281828459045], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 4, 4, 1.2840254166877414], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 4, 4, 1.2840254166877414], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 3, 3, 1.3956124250860895], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 1, 1, 2.718281828459045], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 7, 6, 1.1535649948951077], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 6, 2, 1.1813604128656459], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 24, 2, 1.0425469051899914], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 59, 1, 2.718281828459045], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 6, 2, 1.1813604128656459], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 42, 9, 1.0281671774377144]]
[array([0.99999873, 0.99999723, 0.99999722, 0.99999722, 0.99999644,
       0.99999658, 0.99999685, 0.99999707, 0.99999854, 0.99999928,
       0.99999711, 0.99999919]), array([0.99999676, 0.99999669, 0.99999816, 0.9999974 , 0.99999763,
       0.99999799, 0.99999771, 0.99999719, 0.99999854, 0.99999928,
       0.99999734, 0.999

TypeError: '>' not supported between instances of 'list' and 'float'

In [28]:

train_data = lgb.Dataset(x, label=y)

param = {
    'device_type': 'gpu',
    'objective': 'binary',
    'boosting': 'gbdt',
    "verbosity": -1,
    # 'metric': 'kullback_leibler',
    'metric': 'auc',
    # 'is_unbalance': True,
}
tuner = optuna_lgb.LightGBMTunerCV(param,
                                   train_data,
                                   stratified=False,
                                   verbose_eval=100,
                                   early_stopping_rounds=100,
                                   return_cvbooster=True,
                                   folds=KFold(n_splits=3, shuffle=True))
tuner.run()
bst = tuner.get_best_booster()
bst.save_model('model.txt')


s trial 10 with value: 1.0.
num_leaves, val_score: 1.000000:  80%|########  | 16/20 [02:24<00:39,  9.79s/it][100]	cv_agg's auc: 0.999566 + 0.000613983
[I 2021-01-22 16:34:34,223] Trial 22 finished with value: 0.9999983040959476 and parameters: {'num_leaves': 256}. Best is trial 10 with value: 1.0.
bagging, val_score: 1.000000:  60%|######    | 6/10 [00:43<00:28,  7.12s/it][100]	cv_agg's auc: 0.999532 + 0.000661236
[I 2021-01-22 16:36:02,078] Trial 32 finished with value: 0.9997836642049748 and parameters: {'bagging_fraction': 0.8099374448146474, 'bagging_freq': 6}. Best is trial 30 with value: 0.9999777774358921.
bagging, val_score: 1.000000:  80%|########  | 8/10 [00:58<00:14,  7.12s/it][100]	cv_agg's auc: 0.999504 + 0.000701875
[I 2021-01-22 16:36:16,367] Trial 34 finished with value: 0.9999459808941175 and parameters: {'bagging_fraction': 0.6962677416835248, 'bagging_freq': 3}. Best is trial 30 with value: 0.9999777774358921.
regularization_factors, val_score: 1.000000:  55%|#####5 

In [29]:
msg = [
    'HEEERRREESSS SUISEI~~~', 'wwwww', 'なにそれ', 'YABE', 'やばい', '草', 'lol why',
    'クソクソクソ', '死ね死ね死ね死ね死ね死ね死ね死ね死ね死ね死ね死ね',
    'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', 'きもきもきも',
    'パピオン加藤最強!!パピオン加藤最強!!!パピオン加藤最強!!パピオン加藤最強!!!'
]
bwt = np.nan
test_data = [[
    0.0, 0.0, 0.0, 0.0, 0.0, 6,
    len(x),
    len(set(x)),
    balanced_bwt_rl_entropy([x])
] for x in msg]
print(test_data)
test_df = pd.DataFrame(test_data,
                       columns=[
                           'isVerified', 'isOwner', 'isModerator', 'isMember',
                           'isSuperchat', 'authorLength', 'messageLength',
                           'messageUniqueness', 'bwtrl'
                       ])
embeds = model.encode(msg)
emb_columns = ['emb_' + str(i) for i in range(embeds.shape[1])]
test_df[emb_columns] = embeds
test_df['isMember'] = test_df['isMember']
test_df['isSuperchat'] = test_df['isSuperchat']
sam = data[data['spam'] == 1.0].sample(10)
val_pred = bst.predict(test_df)
print(val_pred)
print((val_pred > 0.65).astype(int))

[[0.0, 0.0, 0.0, 0.0, 0.0, 6, 22, 8, 1.0799589994281866], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 5, 1, 2.718281828459045], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 4, 4, 1.2840254166877414], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 4, 4, 1.2840254166877414], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 3, 3, 1.3956124250860895], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 1, 1, 2.718281828459045], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 7, 6, 1.1535649948951077], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 6, 2, 1.1813604128656459], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 24, 2, 1.0425469051899914], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 59, 1, 2.718281828459045], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 6, 2, 1.1813604128656459], [0.0, 0.0, 0.0, 0.0, 0.0, 6, 42, 9, 1.0281671774377144]]
[array([0.99992939, 0.9997595 , 0.99984053, 0.9997621 , 0.99937809,
       0.99983484, 0.99958312, 0.99974632, 0.99988421, 0.99994937,
       0.99942193, 0.99993532]), array([0.9999371 , 0.99958398, 0.99990516, 0.99970879, 0.99888799,
       0.99965583, 0.99935672, 0.99965481, 0.99961416, 0.99996899,
       0.9994327 , 0.999

TypeError: '>' not supported between instances of 'list' and 'float'